<h2>Lab3: Робота з файлами в Spark Streaming</h2>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, col, desc
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, DoubleType    
import os

os.environ['SPARK_HOME'] = "/home/zaranik/.sdkman/candidates/spark/current"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python3'

Створення Spark-сессії

In [ ]:
spark = SparkSession.builder \
    .appName("Soybean CSV Streaming") \
    .getOrCreate()

Задання схеми даних

In [ ]:
schema = StructType([
    StructField("entity", StringType(), True),
    StructField("code", StringType(), True),
    StructField("year", IntegerType(), True),
    StructField("human_food", DoubleType(), True),
    StructField("animal_feed", DoubleType(), True),
    StructField("processed", DoubleType(), True),
])

Визначення директорії, де будуть знаходитися файли даних

In [ ]:
csv_directory = os.path.join("./csv_lab3/")

Читання файлу як stream

In [ ]:
csv_stream = spark.readStream \
    .option("header", "true") \
    .schema(schema) \
    .csv(csv_directory)

Виведення стріма у консоль

In [ ]:
query = csv_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", False) \
    .start()

Розділення даних на слова

In [ ]:
lines = csv_stream.selectExpr("concat_ws(' ', *) as value")
words = lines.select(explode(split(col("value"), " ")).alias("word"))

Підрахунок елементів у стрімі

In [ ]:
word_counts = words.groupBy("word").count()
sorted_word_counts = word_counts.orderBy(desc("count"))

word_count_query = sorted_word_counts.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()
word_count_query.awaitTermination()

In [ ]:
spark.stop()